**GENERADOR DE FEEDBACK AMB IA GENERATIVA**

In [1]:
from src.core import OPENAI_KEY, CANVAS_BASE_URL, CANVAS_TOKEN, DB_FILEPATH
from src.controllers import FeedbackController
from src.services import ServiceManager, OpenAIService, CanvasService, SQLite3Service

In [2]:
openai_service = OpenAIService(OPENAI_KEY, "gpt-3.5-turbo")
canvas_service = CanvasService(CANVAS_BASE_URL, CANVAS_TOKEN)
sqlite_service = SQLite3Service(DB_FILEPATH)

service_manager = ServiceManager(openai_service, canvas_service, sqlite_service)
feedback_controller = FeedbackController(service_manager)

course_id = "37205"
assignment_id = "434714"
user_id = "300516"

In [3]:
data = feedback_controller.data

course = next((course for course in data if course.id == course_id), None)
assignment = next((assignment for assignment in course.assignments if assignment.id == assignment_id), None)
rubric = assignment.rubric
submission = next((submission for submission in assignment.submissions if submission.user_id == user_id), None)
assessments = submission.assessments
attachment_content = feedback_controller.get_attachment_content(submission.attachment, ["Sara Amzieb"])

data_content = {
    "course": course,
    "assignment": assignment,
    "rubric": rubric,
    "submission": submission,
    "assessments": assessments,
    "attachment_content": attachment_content
} 

In [4]:
feedback_controller.clear_prompt(['user'])

In [5]:
feedback_controller.generate_prompt(**data_content)

Genera el feedback del lliurament avaluable d'un estudiant universitari, de l'activitat Lliurament PAC1 (v1) de l'assignatura Prova AD PoC IA, basat en els criteris de la rúbrica i l'entrega de l'estudiant.

Criteris de la rúbrica:
"""

    - Criteri: Preguntes de disseny
    - Descripció del criteri: Preguntes de disseny
    - Classificació assolida per l'estudiant: Excel·lent
    - Descripció de la classificació: El criteri es compleix de manera excel·lent.
    - Nota: 20.0 / 20.0

    - Criteri: Benchmarking
    - Descripció del criteri: Benchmarking
    - Classificació assolida per l'estudiant: Notable
    - Descripció de la classificació: El criteri es compleix de forma encertada.
    - Nota: 16.0 / 20.0

    - Criteri: Anàlisi heurística
    - Descripció del criteri: Anàlisi heurística
    - Classificació assolida per l'estudiant: Notable
    - Descripció de la classificació: El criteri es compleix de forma encertada.
    - Nota: 8.0 / 10.0

    - Criteri: Arquitectura de la info

In [6]:
feedback_controller.generate_response()